In [ ]:
import pandas as pd
import numpy as np
import os
import os.path as osp

import torchio as tio
from pathlib import Path
from scipy.interpolate import PchipInterpolator, interp1d
from matplotlib import pyplot as plt
from scipy.stats import norm

In [ ]:
internal_data_pths = [
                      './datasets/LN_classify/Fudan_HN_LN_22-23_RAD/Fudan_HN_LN_20231204_patches',
                      ]

external_data_pths = ['./datasets/LN_classify/Fudan_HN_LN_22-23_RAD/CGMH/CGMH_2024_patches',
                      './datasets/LN_classify/Fudan_HN_LN_22-23_RAD/TCGA/TCGA-HNSC_selected_patches',
                      './datasets/LN_classify/Fudan_HN_LN_22-23_RAD/TCGA/TCGA-fixed-RAD_patches',
                      './datasets/LN_classify/Fudan_HN_LN_22-23_RAD/CGMH_Oral/CGMH_Oral_patches'
                      ]

name_to_sad = {}
for data_idx, crop_pth in enumerate(internal_data_pths + external_data_pths):
    cropfile = osp.join(crop_pth, "cropping_list.csv")
    df = pd.read_csv(cropfile)
    for idx, row in df.iterrows():
        name_to_sad[row['basename'].lower()] = float(row['recist'])

In [ ]:
_MIN_SAD = 5.0
# _MAX_SAD = {'EENT': 21.8850, 'CGMH': 11.25, 'TCGA': 9999, 'CGMH-Oral': 9999, 'Dev': 9999} # q50 limit
_MAX_SAD = {'EENT': 9999, 'CGMH': 9999, 'TCGA': 9999, 'CGMH-Oral': 9999, 'Dev': 9999} # no limit
renji_ln_count = {}
renji_ln_rm = {}
cgmh_ln_count = {}
cgmh_ln_rm = {}
tcga_ln_count = {}
tcga_ln_rm = {}

# renji_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_all/Fudan_HN_LN_20231204/2023-08_LN_data_reorganize_external"
renji_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_RAD/Fudan_HN_LN_20231204_patches/Ext"
# cgmg_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_all/CGMH/CGMH_2024_reoriented"
cgmh_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_RAD/CGMH/CGMH_2024_patches"
# tcga_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_all/TCGA/TCGA-HNSC_selected_reori_labeled"
tcga_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_RAD/TCGA/TCGA-HNSC_selected_patches"
tcga_rad_add_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_RAD/TCGA/TCGA-fixed-RAD_patches"

for p in Path(renji_pth).rglob("*_mask.nii.gz"):
    basename = osp.basename(p).replace("_mask.nii.gz", "")
    patient = basename.split('_ins')[0]
    ln_id = int(basename.split('_ins')[-1].split('_')[0])
    if '_pos' in basename:
        sad = name_to_sad[basename.lower().split('_pos')[0]]
    else:
        sad = name_to_sad[basename.lower().split('_neg')[0]]
    
    if patient not in renji_ln_count:
        renji_ln_count[patient] = []
        renji_ln_rm[patient] = []
    
    if sad >= 5 or '_pos' in basename:
        renji_ln_count[patient].append(ln_id)
    else:
        renji_ln_rm[patient].append(ln_id)

for p in Path(cgmh_pth).rglob("*_mask.nii.gz"):
    basename = osp.basename(p).replace("_mask.nii.gz", "")
    patient = basename.split('_ins')[0]
    ln_id = int(basename.split('_ins')[-1].split('_')[0])
    if '_pos' in basename:
        sad = name_to_sad[basename.lower().split('_pos')[0]]
    else:
        sad = name_to_sad[basename.lower().split('_neg')[0]]
    
    if patient not in cgmh_ln_count:
        cgmh_ln_count[patient] = []
        cgmh_ln_rm[patient] = []
    
    if sad >= 5 or '_pos' in basename:
        cgmh_ln_count[patient].append(ln_id)
    else:
        cgmh_ln_rm[patient].append(ln_id)
    
for p in Path(tcga_pth).rglob("*_mask.nii.gz"):
    basename = osp.basename(p).replace("_mask.nii.gz", "")
    patient = basename.split('_ins')[0]
    if '_pos' in basename:
        sad = name_to_sad[basename.lower().split('_pos')[0]]
    else:
        sad = name_to_sad[basename.lower().split('_neg')[0]]
    
    if patient not in tcga_ln_count:
        tcga_ln_count[patient] = []
        tcga_ln_rm[patient] = []
    
    if sad >= 5 or '_pos' in basename:
        tcga_ln_count[patient].append(ln_id)
    else:
        tcga_ln_rm[patient].append(ln_id)

# RAD added TCGA
for p in Path(tcga_rad_add_pth).rglob("*_mask.nii.gz"):
    basename = osp.basename(p).replace("_mask.nii.gz", "")
    patient = basename.split('_ins')[0]
    if '_pos' in basename:
        sad = name_to_sad[basename.lower().split('_pos')[0]]
    else:
        sad = name_to_sad[basename.lower().split('_neg')[0]]
    
    if patient not in tcga_ln_count:
        tcga_ln_count[patient] = []
        tcga_ln_rm[patient] = []
    
    if sad >= 5 or '_pos' in basename:
        tcga_ln_count[patient].append(ln_id)
    else:
        tcga_ln_rm[patient].append(ln_id)

#### Load annotations

In [ ]:
renji_anno_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_RAD/Fudan_HN_LN_20231204/headNeck_LN_label_infor(7).csv"
cgmh_anno_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_RAD/CGMH/headNeck_LN_label_infor.csv"
tcga_anno_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_RAD/TCGA/headNeck_LN_label_infor.csv"
df_renji = pd.read_csv(renji_anno_pth)
df_cgmh = pd.read_csv(cgmh_anno_pth)
df_tcga = pd.read_csv(tcga_anno_pth)

In [ ]:
# Load reader 1 and reader 2's annotation
rd1_renji_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_RAD/ReaderStudy/reader1_renji_remapped.csv"
rd1_cgmh_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_RAD/ReaderStudy/reader1_cgmh.csv"
rd1_tcga_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_RAD/ReaderStudy/reader1_tcga_remapped.csv"
df_rd1_renji = pd.read_csv(rd1_renji_pth)
df_rd1_cgmh = pd.read_csv(rd1_cgmh_pth)
df_rd1_tcga = pd.read_csv(rd1_tcga_pth)

rd2_renji_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_RAD/ReaderStudy/reader2_renji_remapped.csv"
rd2_cgmh_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_RAD/ReaderStudy/reader2_cgmh.csv"
rd2_tcga_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_RAD/ReaderStudy/reader2_tcga_remapped.csv"
df_rd2_renji = pd.read_csv(rd2_renji_pth)
df_rd2_cgmh = pd.read_csv(rd2_cgmh_pth)
df_rd2_tcga = pd.read_csv(rd2_tcga_pth)

# rd3_renji_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_RAD/ReaderStudy/reader3_sn_renji_remapped.csv"
# rd3_cgmh_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_RAD/ReaderStudy/reader3_sn_cgmh.csv"
# rd3_tcga_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_RAD/ReaderStudy/reader3_sn_tcga_remapped.csv"
# df_rd3_renji = pd.read_csv(rd3_renji_pth)
# df_rd3_cgmh = pd.read_csv(rd3_cgmh_pth)
# df_rd3_tcga = pd.read_csv(rd3_tcga_pth)

rd4_renji_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_RAD/ReaderStudy/reader4_dd_renji_remapped.csv"
rd4_cgmh_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_RAD/ReaderStudy/reader4_dd_cgmh.csv"
rd4_tcga_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_RAD/ReaderStudy/reader4_dd_tcga_remapped.csv"
df_rd4_renji = pd.read_csv(rd4_renji_pth)
df_rd4_cgmh = pd.read_csv(rd4_cgmh_pth)
df_rd4_tcga = pd.read_csv(rd4_tcga_pth)

rd5_renji_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_RAD/ReaderStudy/reader5_xbb_renji_remapped.csv"
rd5_cgmh_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_RAD/ReaderStudy/reader5_xbb_cgmh.csv"
rd5_tcga_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_RAD/ReaderStudy/reader5_xbb_tcga_remapped.csv"
df_rd5_renji = pd.read_csv(rd5_renji_pth)
df_rd5_cgmh = pd.read_csv(rd5_cgmh_pth)
df_rd5_tcga = pd.read_csv(rd5_tcga_pth)

rd6_renji_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_RAD/ReaderStudy/reader6_albert_renji_remapped.csv"
rd6_cgmh_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_RAD/ReaderStudy/reader6_albert_cgmh.csv"
rd6_tcga_pth = "./datasets/LN_classify/Fudan_HN_LN_22-23_RAD/ReaderStudy/reader6_albert_tcga_remapped.csv"
df_rd6_renji = pd.read_csv(rd6_renji_pth)
df_rd6_cgmh = pd.read_csv(rd6_cgmh_pth)
df_rd6_tcga = pd.read_csv(rd6_tcga_pth)

In [8]:
def calc_perf(df_reader, df_anno, type):
    total_gt_meta, total_gt_ENE = 0, 0
    total_hit_meta, total_hit_ENE = 0, 0
    total_tn_meta, total_tn_ENE, total_tn_ENEvsPos = 0, 0, 0
    total_pred_meta, total_pred_ENE = 0, 0
    total_annotations = 0
    for idx, reader_row in df_reader.iterrows():
        patient = reader_row["file_name"].replace(".nii.gz", "")
        if type == 'renji':
            patient_annotations = len(renji_ln_count[patient])
        elif type == 'cgmh':
            if 'CGMH_Larynx20240117_anon_025' in patient:
                continue
            patient_annotations = len(cgmh_ln_count[patient])
        elif type == 'tcga':
            patient_annotations = len(tcga_ln_count[patient])
        else:
            raise ValueError("Invalid type")
        total_annotations += patient_annotations
            
        row_meta_preds = reader_row["positive_LN_id"]
        if isinstance(row_meta_preds, pd.Series):
            row_meta_preds = row_meta_preds.values[0]
        
        if isinstance(row_meta_preds, float) and np.isnan(row_meta_preds):
            meta_preds = []
        elif isinstance(row_meta_preds, (int, float)):
            meta_preds = [int(row_meta_preds)]
        elif isinstance(row_meta_preds, str) and row_meta_preds != "无转移" and row_meta_preds != "":
            meta_preds = [int(p) for p in row_meta_preds.rstrip(',').split(",")]
        elif row_meta_preds == "无转移" or row_meta_preds == "":
            meta_preds = []
        else:
            raise ValueError("No predicted found")
        
        # remove not selected LNs (e.g., <5mm negative and >LIMIT ENEs)
        if type == 'renji':
            meta_preds = [p for p in meta_preds if p not in renji_ln_rm[patient]]
        elif type == 'cgmh':
            meta_preds = [p for p in meta_preds if p not in cgmh_ln_rm[patient]]
        elif type == 'tcga':
            meta_preds = [p for p in meta_preds if p not in tcga_ln_rm[patient]]
        
        row_ene_preds = reader_row["ENE_LN_id"]
        if isinstance(row_ene_preds, pd.Series):
            row_ene_preds = row_ene_preds.values[0]
          
        if isinstance(row_ene_preds, float) and np.isnan(row_ene_preds):
            ene_preds = []
        elif isinstance(row_ene_preds, (int, float)):
            ene_preds = [int(row_ene_preds)]
        elif isinstance(row_ene_preds, str):
            ene_preds = [int(p) for p in row_ene_preds.rstrip(',').split(",")]
        else:
            ene_preds = []
        
        # remove not selected LNs (e.g., <5mm negative and >LIMIT ENEs)
        if type == 'renji':
            ene_preds = [p for p in ene_preds if p not in renji_ln_rm[patient]]
        elif type == 'cgmh':
            ene_preds = [p for p in ene_preds if p not in cgmh_ln_rm[patient]]
        elif type == 'tcga':
            ene_preds = [p for p in ene_preds if p not in tcga_ln_rm[patient]]
        
        anno_row = df_anno[df_anno["file_name"] == patient]
        if anno_row.empty:
            print("{} is not matched any annotation".format(patient))
            continue
        
        row_anno_meta = anno_row["positive_LN_id"]
        if isinstance(row_anno_meta, pd.Series):
            row_anno_meta = row_anno_meta.values[0]
        
        if isinstance(row_anno_meta, float) and np.isnan(row_anno_meta):
            anno_meta = []
        elif isinstance(row_anno_meta, (int, float)):
            anno_meta = [int(row_anno_meta)]
        elif isinstance(row_anno_meta, str) and row_anno_meta != "无转移":
            anno_meta = [int(p) for p in row_anno_meta.rstrip(',').split(",")]
        elif row_anno_meta == "无转移":
            anno_meta = []
        else:
            raise ValueError("No annotation found")
        
        # remove not selected LNs (e.g., <5mm negative and >LIMIT ENEs)
        if type == 'renji':
            anno_meta = [p for p in anno_meta if p not in renji_ln_rm[patient]]
        elif type == 'cgmh':
            anno_meta = [p for p in anno_meta if p not in cgmh_ln_rm[patient]]
        elif type == 'tcga':
            anno_meta = [p for p in anno_meta if p not in tcga_ln_rm[patient]]
        
        row_anno_ene = anno_row["ENE_LN_id"]
        if isinstance(row_anno_ene, pd.Series):
            row_anno_ene = row_anno_ene.values[0]
          
        if isinstance(row_anno_ene, float) and np.isnan(row_anno_ene):
            anno_ene = []
        elif isinstance(row_anno_ene, (int, float)):
            anno_ene = [int(row_anno_ene)]
        elif isinstance(row_anno_ene, str):
            anno_ene = [int(p) for p in row_anno_ene.rstrip(',').split(",")]
        else:
            anno_ene = []
        
        # remove not selected LNs (e.g., <5mm negative and >LIMIT ENEs)
        if type == 'renji':
            anno_ene = [p for p in anno_ene if p not in renji_ln_rm[patient]]
        elif type == 'cgmh':
            anno_ene = [p for p in anno_ene if p not in cgmh_ln_rm[patient]]
        elif type == 'tcga':
            anno_ene = [p for p in anno_ene if p not in tcga_ln_rm[patient]]
            
        meta_preds = set(meta_preds)
        ene_preds = set(ene_preds)
        anno_meta = set(anno_meta)
        anno_ene = set(anno_ene)
        fps_meta = len(set(meta_preds) - set(anno_meta))
        fps_ene = len(set(ene_preds) - set(anno_ene))
        fps_ene_in_pos = len(set(ene_preds).intersection(anno_meta) - set(anno_ene))
        num_gt_meta = len(anno_meta)
        num_gt_ene = len(anno_ene)
        

        total_hit_meta += len(meta_preds.intersection(anno_meta))
        total_hit_ENE += len(ene_preds.intersection(anno_ene))
        total_pred_meta += len(meta_preds)
        total_pred_ENE += len(ene_preds)
        total_gt_meta += len(anno_meta)
        total_gt_ENE += len(anno_ene)
        total_tn_meta += patient_annotations - num_gt_meta - fps_meta
        total_tn_ENE += patient_annotations - num_gt_ene - fps_ene
        total_tn_ENEvsPos += num_gt_meta - num_gt_ene - fps_ene_in_pos
        
    meta_sens = total_hit_meta / (total_gt_meta + 1e-6)
    meta_spec = total_tn_meta / (total_annotations - total_gt_meta)
    ene_sens = total_hit_ENE / (total_gt_ENE + 1e-6)
    ene_spec = total_tn_ENE / (total_annotations - total_gt_ENE + 1e-6)
    
    meta_acc = (total_hit_meta + total_tn_meta) / total_annotations
    ene_acc = (total_hit_ENE + total_tn_ENE) / total_annotations
    
    enevspos_sens = total_hit_ENE / total_gt_ENE
    enevspos_spec = total_tn_ENEvsPos / (total_gt_meta - total_gt_ENE + 1e-6)
    enevspos_acc = (total_hit_ENE + total_tn_ENEvsPos) / total_gt_meta
    # print("ENE Vs Pos: Sens: {:.4f}, Spec: {:.4f}".format(enevspos_sens, enevspos_spec))
    
    res = {}
    res['meta_sens'] = meta_sens
    res['meta_spec'] = meta_spec
    res['meta_acc'] = meta_acc
    res['meta_precision'] = total_hit_meta / (total_pred_meta + 1e-6)
    res['meta_f1'] = 2 * res['meta_precision'] * meta_sens / (res['meta_precision'] + meta_sens + 1e-6)
    res['ene_sens'] = ene_sens
    res['ene_spec'] = ene_spec
    res['ene_acc'] = ene_acc
    res['ene_precision'] = total_hit_ENE / (total_pred_ENE + 1e-6)
    res['ene_f1'] = 2 * res['ene_precision'] * ene_sens / (res['ene_precision'] + ene_sens + 1e-6)
    res['total_gt_meta'] = total_gt_meta
    res['total_gt_ENE'] = total_gt_ENE
    res['total_neg_meta'] = total_annotations - total_gt_meta
    res['total_neg_ENE'] = total_annotations - total_gt_ENE
    res['enevspos_sens'] = enevspos_sens
    res['enevspos_spec'] = enevspos_spec
    res['enevspos_acc'] = enevspos_acc
    
    # return meta_sens, meta_spec, meta_acc, ene_sens, ene_spec, ene_acc, (total_gt_meta, total_annotations - total_gt_meta, total_gt_ENE, total_annotations - total_gt_ENE) 
    return res
        
        

In [9]:
def calc_patient_level_perf(df_reader, df_anno, type):
    total_gt_meta, total_gt_ENE = 0, 0
    total_hit_meta, total_hit_ENE = 0, 0
    total_tn_meta, total_tn_ENE = 0, 0
    total_pred_meta, total_pred_ENE = 0, 0
    total_patients = 0
    all_patient_ene_preds, all_anno_ene = [], []
    patient_list = []
    for idx, reader_row in df_reader.iterrows():
        patient = reader_row["file_name"].replace(".nii.gz", "")
        if type == 'renji':
            patient_annotations = len(renji_ln_count[patient])
        elif type == 'cgmh':
            if 'CGMH_Larynx20240117_anon_025' in patient:
                continue
            patient_annotations = len(cgmh_ln_count[patient])
        elif type == 'tcga':
            patient_annotations = len(tcga_ln_count[patient])
        else:
            raise ValueError("Invalid type")
        
        patient_list.append(patient)
        total_patients += 1
            
        row_meta_preds = reader_row["positive_LN_id"]
        if isinstance(row_meta_preds, pd.Series):
            row_meta_preds = row_meta_preds.values[0]
        
        if isinstance(row_meta_preds, float) and np.isnan(row_meta_preds):
            meta_preds = []
        elif isinstance(row_meta_preds, (int, float)):
            meta_preds = [int(row_meta_preds)]
        elif isinstance(row_meta_preds, str) and row_meta_preds != "无转移" and row_meta_preds != "":
            meta_preds = [int(p) for p in row_meta_preds.rstrip(',').split(",")]
        elif row_meta_preds == "无转移" or row_meta_preds == "":
            meta_preds = []
        else:
            raise ValueError("No predicted found")
        
        # remove not selected LNs (e.g., <5mm negative and >LIMIT ENEs)
        if type == 'renji':
            meta_preds = [p for p in meta_preds if p not in renji_ln_rm[patient]]
        elif type == 'cgmh':
            meta_preds = [p for p in meta_preds if p not in cgmh_ln_rm[patient]]
        elif type == 'tcga':
            meta_preds = [p for p in meta_preds if p not in tcga_ln_rm[patient]]
        
        row_ene_preds = reader_row["ENE_LN_id"]
        if isinstance(row_ene_preds, pd.Series):
            row_ene_preds = row_ene_preds.values[0]
          
        if isinstance(row_ene_preds, float) and np.isnan(row_ene_preds):
            ene_preds = []
        elif isinstance(row_ene_preds, (int, float)):
            ene_preds = [int(row_ene_preds)]
        elif isinstance(row_ene_preds, str):
            ene_preds = [int(p) for p in row_ene_preds.rstrip(',').split(",")]
        else:
            ene_preds = []
        
        # remove not selected LNs (e.g., <5mm negative and >LIMIT ENEs)
        if type == 'renji':
            ene_preds = [p for p in ene_preds if p not in renji_ln_rm[patient]]
        elif type == 'cgmh':
            ene_preds = [p for p in ene_preds if p not in cgmh_ln_rm[patient]]
        elif type == 'tcga':
            ene_preds = [p for p in ene_preds if p not in tcga_ln_rm[patient]]
        
        anno_row = df_anno[df_anno["file_name"] == patient]
        if anno_row.empty:
            print("{} is not matched any annotation".format(patient))
            continue
        
        row_anno_meta = anno_row["positive_LN_id"]
        if isinstance(row_anno_meta, pd.Series):
            row_anno_meta = row_anno_meta.values[0]
        
        if isinstance(row_anno_meta, float) and np.isnan(row_anno_meta):
            anno_meta = []
        elif isinstance(row_anno_meta, (int, float)):
            anno_meta = [int(row_anno_meta)]
        elif isinstance(row_anno_meta, str) and row_anno_meta != "无转移":
            anno_meta = [int(p) for p in row_anno_meta.rstrip(',').split(",")]
        elif row_anno_meta == "无转移":
            anno_meta = []
        else:
            raise ValueError("No annotation found")
        
        # remove not selected LNs (e.g., <5mm negative and >LIMIT ENEs)
        if type == 'renji':
            anno_meta = [p for p in anno_meta if p not in renji_ln_rm[patient]]
        elif type == 'cgmh':
            anno_meta = [p for p in anno_meta if p not in cgmh_ln_rm[patient]]
        elif type == 'tcga':
            anno_meta = [p for p in anno_meta if p not in tcga_ln_rm[patient]]
        
        row_anno_ene = anno_row["ENE_LN_id"]
        if isinstance(row_anno_ene, pd.Series):
            row_anno_ene = row_anno_ene.values[0]
          
        if isinstance(row_anno_ene, float) and np.isnan(row_anno_ene):
            anno_ene = []
        elif isinstance(row_anno_ene, (int, float)):
            anno_ene = [int(row_anno_ene)]
        elif isinstance(row_anno_ene, str):
            anno_ene = [int(p) for p in row_anno_ene.rstrip(',').split(",")]
        else:
            anno_ene = []
        
        # remove not selected LNs (e.g., <5mm negative and >LIMIT ENEs)
        if type == 'renji':
            anno_ene = [p for p in anno_ene if p not in renji_ln_rm[patient]]
        elif type == 'cgmh':
            anno_ene = [p for p in anno_ene if p not in cgmh_ln_rm[patient]]
        elif type == 'tcga':
            anno_ene = [p for p in anno_ene if p not in tcga_ln_rm[patient]]
            
        patient_meta_preds = len(meta_preds) > 0
        patient_ene_preds = len(ene_preds) > 0
        anno_meta = len(anno_meta) > 0
        anno_ene = len(anno_ene) > 0

        total_hit_meta += 1 if (patient_meta_preds == anno_meta and anno_meta) else 0
        total_hit_ENE += 1 if (patient_ene_preds == anno_ene and anno_ene) else 0
        total_pred_meta += patient_meta_preds * 1.0
        total_pred_ENE += patient_ene_preds * 1.0
        total_gt_meta += anno_meta * 1.0
        total_gt_ENE += anno_ene * 1.0
        total_tn_meta += (1 - anno_meta) * (1 - patient_meta_preds)
        total_tn_ENE += (1 - anno_ene) * (1 - patient_ene_preds)

        all_anno_ene.append(anno_ene)
        all_patient_ene_preds.append(patient_ene_preds)
        
    meta_sens = total_hit_meta / (total_gt_meta + 1e-6)
    meta_spec = total_tn_meta / (total_patients - total_gt_meta)
    ene_sens = total_hit_ENE / (total_gt_ENE + 1e-6)
    ene_spec = total_tn_ENE / (total_patients - total_gt_ENE + 1e-6)
    
    meta_acc = (total_hit_meta + total_tn_meta) / total_patients
    ene_acc = (total_hit_ENE + total_tn_ENE) / total_patients
    
    res = {}
    res['meta_sens'] = meta_sens
    res['meta_spec'] = meta_spec
    res['meta_acc'] = meta_acc
    res['meta_precision'] = total_hit_meta / (total_pred_meta + 1e-6)
    res['meta_f1'] = 2 * res['meta_precision'] * meta_sens / (res['meta_precision'] + meta_sens + 1e-6)
    res['ene_sens'] = ene_sens
    res['ene_spec'] = ene_spec
    res['ene_acc'] = ene_acc
    res['ene_precision'] = total_hit_ENE / (total_pred_ENE + 1e-6)
    res['ene_f1'] = 2 * res['ene_precision'] * ene_sens / (res['ene_precision'] + ene_sens + 1e-6)
    res['total_gt_meta'] = total_gt_meta
    res['total_gt_ENE'] = total_gt_ENE
    res['total_neg_meta'] = total_patients - total_gt_meta
    res['total_neg_ENE'] = total_patients - total_gt_ENE

    sort_idx = np.argsort(patient_list).tolist()
    all_patient_ene_preds = np.array(all_patient_ene_preds)[sort_idx]
    all_anno_ene = np.array(all_anno_ene)[sort_idx]
    res['patient_ene_preds'] = all_patient_ene_preds
    res['anno_ene'] = all_anno_ene
    
    # return meta_sens, meta_spec, meta_acc, ene_sens, ene_spec, ene_acc, (total_gt_meta, total_annotations - total_gt_meta, total_gt_ENE, total_annotations - total_gt_ENE) 
    return res
        
        

In [10]:
def auc_from_roc_points(fpr, tpr):
    # Ensure fpr and tpr are sorted by fpr
    sorted_indices = np.argsort(fpr)
    sorted_fpr = fpr[sorted_indices]
    sorted_tpr = tpr[sorted_indices]
    return np.trapz(sorted_tpr, sorted_fpr)

def auc_ci_from_roc_points(fpr, tpr, n_pos, n_neg, alpha=0.95):
    auc = auc_from_roc_points(fpr, tpr)
    
    # Calculate standard error
    q1 = auc / (2 - auc)
    q2 = 2 * auc**2 / (1 + auc)
    se = np.sqrt((auc * (1 - auc) + (n_pos - 1) * (q1 - auc**2) + (n_neg - 1) * (q2 - auc**2)) / (n_pos * n_neg))
    
    # Calculate confidence interval
    z = norm.ppf(1 - (1 - alpha) / 2)
    ci_lower = auc - z * se
    ci_upper = auc + z * se
    
    return auc, ci_lower, ci_upper

def bootstrap_auc_ci(fpr, tpr, n_bootstraps=2000, alpha=0.95, random_state=None):
    if random_state is not None:
        np.random.seed(random_state)
    
    auc_original = auc_from_roc_points(fpr, tpr)
    bootstrapped_aucs = []

    for _ in range(n_bootstraps):
        # Resample FPR and TPR points with replacement
        indices = np.random.randint(0, len(fpr), len(fpr))
        if len(np.unique(fpr[indices])) < 2 or len(np.unique(tpr[indices])) < 2:
            # Skip this sample because there are not enough unique points
            continue

        bootstrapped_fpr = fpr[indices]
        bootstrapped_tpr = tpr[indices]
        bootstrapped_auc = auc_from_roc_points(bootstrapped_fpr, bootstrapped_tpr)
        bootstrapped_aucs.append(bootstrapped_auc)

    sorted_aucs = np.sort(bootstrapped_aucs)
    lower_percentile = (1.0 - alpha) / 2.0 * 100
    upper_percentile = (1.0 + alpha) / 2.0 * 100
    ci_lower = np.percentile(sorted_aucs, lower_percentile)
    ci_upper = np.percentile(sorted_aucs, upper_percentile)
    
    return auc_original, ci_lower, ci_upper

def calc_physician_auc(sensitivity, specificity, num_pos, num_neg, draw=False):
    fpr = 1 - specificity

    # Define the initial points
    points = np.array([
        [0, 0],               # (0, 0)
        [fpr, sensitivity],   # (1 - specificity, sensitivity)
        [1, 1]                # (1, 1)
    ])

    # Number of points for interpolation
    num_points = 100

    # Generate x values (False Positive Rates) for interpolation
    x_vals = np.linspace(0, 1, num_points)

    # Perform linear interpolation
    # y_vals = PchipInterpolator(points[:, 0], points[:, 1])(x_vals)
    linear_interp = interp1d(points[:, 0], points[:, 1], kind='linear')
    y_vals = linear_interp(x_vals)

    if draw:
    # Plotting the ROC curve
        plt.plot(x_vals, y_vals, linestyle='-', color='b', label='Linear Interpolation')
        plt.scatter(points[:, 0], points[:, 1], color='r', zorder=5, label='Original Points')
        plt.xlabel('False Positive Rate (1 - Specificity)')
        plt.ylabel('True Positive Rate (Sensitivity)')
        plt.title('ROC Curve with Linear Interpolation')
        plt.legend()
        plt.grid(True)
        plt.show()

    # raw_auc, ci_lo, ci_hi = bootstrap_auc_ci(x_vals, y_vals)
    raw_auc, ci_lo, ci_hi = auc_ci_from_roc_points(x_vals, y_vals, num_pos, num_neg)
    print("AUC: {} ({}-{})".format(raw_auc, ci_lo, ci_hi))

print("Reader1-WJ:")
res = calc_perf(df_rd1_renji, df_renji, type='renji')
print("Renji meta sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['meta_sens'], res['meta_spec'], res['meta_f1'], res['meta_acc']))
calc_physician_auc(res['meta_sens'], res['meta_spec'], res['total_gt_meta'], res['total_neg_meta'])
print("Renji ENE sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['ene_sens'], res['ene_spec'], res['ene_f1'], res['ene_acc']))
print("ENE+ Vs. ENE- sensitivity {:.4f}, specificity {:.4f}, acc {:.4f}".format(res['enevspos_sens'], res['enevspos_spec'], res['enevspos_acc']))
calc_physician_auc(res['ene_sens'], res['ene_spec'], res['total_gt_ENE'], res['total_neg_ENE'])
calc_physician_auc(res['enevspos_sens'], res['enevspos_spec'], res['total_gt_ENE'], res['total_gt_meta'] - res['total_gt_ENE'])
res = calc_perf(df_rd1_cgmh, df_cgmh, type='cgmh')
print("CGMH meta sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['meta_sens'], res['meta_spec'], res['meta_f1'], res['meta_acc']))
calc_physician_auc(res['meta_sens'], res['meta_spec'], res['total_gt_meta'], res['total_neg_meta'])
print("CGMH ENE sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['ene_sens'], res['ene_spec'], res['ene_f1'], res['ene_acc']))
print("ENE+ Vs. ENE- sensitivity {:.4f}, specificity {:.4f}, acc {:.4f}".format(res['enevspos_sens'], res['enevspos_spec'], res['enevspos_acc']))
calc_physician_auc(res['ene_sens'], res['ene_spec'], res['total_gt_ENE'], res['total_neg_ENE'])
calc_physician_auc(res['enevspos_sens'], res['enevspos_spec'], res['total_gt_ENE'], res['total_gt_meta'] - res['total_gt_ENE'])
res = calc_perf(df_rd1_tcga, df_tcga, type='tcga')
print("TCGA meta sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['meta_sens'], res['meta_spec'], res['meta_f1'], res['meta_acc']))
calc_physician_auc(res['meta_sens'], res['meta_spec'], res['total_gt_meta'], res['total_neg_meta'])
print("TCGA ENE sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['ene_sens'], res['ene_spec'], res['ene_f1'], res['ene_acc']))
calc_physician_auc(res['ene_sens'], res['ene_spec'], res['total_gt_ENE'], res['total_neg_ENE'])

print("Reader2-Zhang:")
res = calc_perf(df_rd2_renji, df_renji, type='renji')
print("Renji meta sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['meta_sens'], res['meta_spec'], res['meta_f1'], res['meta_acc']))
calc_physician_auc(res['meta_sens'], res['meta_spec'], res['total_gt_meta'], res['total_neg_meta'])
print("Renji ENE sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['ene_sens'], res['ene_spec'], res['ene_f1'], res['ene_acc']))
print("ENE+ Vs. ENE- sensitivity {:.4f}, specificity {:.4f}, acc {:.4f}".format(res['enevspos_sens'], res['enevspos_spec'], res['enevspos_acc']))
calc_physician_auc(res['ene_sens'], res['ene_spec'], res['total_gt_ENE'], res['total_neg_ENE'])
calc_physician_auc(res['enevspos_sens'], res['enevspos_spec'], res['total_gt_ENE'], res['total_gt_meta'] - res['total_gt_ENE'])
res = calc_perf(df_rd2_cgmh, df_cgmh, type='cgmh')
print("CGMH meta sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['meta_sens'], res['meta_spec'], res['meta_f1'], res['meta_acc']))
calc_physician_auc(res['meta_sens'], res['meta_spec'], res['total_gt_meta'], res['total_neg_meta'])
print("CGMH ENE sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['ene_sens'], res['ene_spec'], res['ene_f1'], res['ene_acc']))
print("ENE+ Vs. ENE- sensitivity {:.4f}, specificity {:.4f}, acc {:.4f}".format(res['enevspos_sens'], res['enevspos_spec'], res['enevspos_acc']))
calc_physician_auc(res['ene_sens'], res['ene_spec'], res['total_gt_ENE'], res['total_neg_ENE'])
calc_physician_auc(res['enevspos_sens'], res['enevspos_spec'], res['total_gt_ENE'], res['total_gt_meta'] - res['total_gt_ENE'])
res = calc_perf(df_rd2_tcga, df_tcga, type='tcga')
print("TCGA meta sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['meta_sens'], res['meta_spec'], res['meta_f1'], res['meta_acc']))
calc_physician_auc(res['meta_sens'], res['meta_spec'], res['total_gt_meta'], res['total_neg_meta'])
print("TCGA ENE sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['ene_sens'], res['ene_spec'], res['ene_f1'], res['ene_acc']))
calc_physician_auc(res['ene_sens'], res['ene_spec'], res['total_gt_ENE'], res['total_neg_ENE'])

print("Reader3-DD:")
res = calc_perf(df_rd4_renji, df_renji, type='renji')
print("Renji meta sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['meta_sens'], res['meta_spec'], res['meta_f1'], res['meta_acc']))
calc_physician_auc(res['meta_sens'], res['meta_spec'], res['total_gt_meta'], res['total_neg_meta'])
print("Renji ENE sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['ene_sens'], res['ene_spec'], res['ene_f1'], res['ene_acc']))
print("ENE+ Vs. ENE- sensitivity {:.4f}, specificity {:.4f}, acc {:.4f}".format(res['enevspos_sens'], res['enevspos_spec'], res['enevspos_acc']))
calc_physician_auc(res['ene_sens'], res['ene_spec'], res['total_gt_ENE'], res['total_neg_ENE'])
calc_physician_auc(res['enevspos_sens'], res['enevspos_spec'], res['total_gt_ENE'], res['total_gt_meta'] - res['total_gt_ENE'])
res = calc_perf(df_rd4_cgmh, df_cgmh, type='cgmh')
print("CGMH meta sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['meta_sens'], res['meta_spec'], res['meta_f1'], res['meta_acc']))
calc_physician_auc(res['meta_sens'], res['meta_spec'], res['total_gt_meta'], res['total_neg_meta'])
print("CGMH ENE sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['ene_sens'], res['ene_spec'], res['ene_f1'], res['ene_acc']))
print("ENE+ Vs. ENE- sensitivity {:.4f}, specificity {:.4f}, acc {:.4f}".format(res['enevspos_sens'], res['enevspos_spec'], res['enevspos_acc']))
calc_physician_auc(res['ene_sens'], res['ene_spec'], res['total_gt_ENE'], res['total_neg_ENE'])
calc_physician_auc(res['enevspos_sens'], res['enevspos_spec'], res['total_gt_ENE'], res['total_gt_meta'] - res['total_gt_ENE'])
res = calc_perf(df_rd4_tcga, df_tcga, type='tcga')
print("TCGA meta sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['meta_sens'], res['meta_spec'], res['meta_f1'], res['meta_acc']))
calc_physician_auc(res['meta_sens'], res['meta_spec'], res['total_gt_meta'], res['total_neg_meta'])
print("TCGA ENE sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['ene_sens'], res['ene_spec'], res['ene_f1'], res['ene_acc']))
calc_physician_auc(res['ene_sens'], res['ene_spec'], res['total_gt_ENE'], res['total_neg_ENE'])

print("Reader4-XBB:")
res = calc_perf(df_rd5_renji, df_renji, type='renji')
print("Renji meta sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['meta_sens'], res['meta_spec'], res['meta_f1'], res['meta_acc']))
calc_physician_auc(res['meta_sens'], res['meta_spec'], res['total_gt_meta'], res['total_neg_meta'])
print("Renji ENE sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['ene_sens'], res['ene_spec'], res['ene_f1'], res['ene_acc']))
print("ENE+ Vs. ENE- sensitivity {:.4f}, specificity {:.4f}, acc {:.4f}".format(res['enevspos_sens'], res['enevspos_spec'], res['enevspos_acc']))
calc_physician_auc(res['ene_sens'], res['ene_spec'], res['total_gt_ENE'], res['total_neg_ENE'])
calc_physician_auc(res['enevspos_sens'], res['enevspos_spec'], res['total_gt_ENE'], res['total_gt_meta'] - res['total_gt_ENE'])
res = calc_perf(df_rd5_cgmh, df_cgmh, type='cgmh')
print("CGMH meta sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['meta_sens'], res['meta_spec'], res['meta_f1'], res['meta_acc']))
calc_physician_auc(res['meta_sens'], res['meta_spec'], res['total_gt_meta'], res['total_neg_meta'])
print("CGMH ENE sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['ene_sens'], res['ene_spec'], res['ene_f1'], res['ene_acc']))
print("ENE+ Vs. ENE- sensitivity {:.4f}, specificity {:.4f}, acc {:.4f}".format(res['enevspos_sens'], res['enevspos_spec'], res['enevspos_acc']))
calc_physician_auc(res['ene_sens'], res['ene_spec'], res['total_gt_ENE'], res['total_neg_ENE'])
calc_physician_auc(res['enevspos_sens'], res['enevspos_spec'], res['total_gt_ENE'], res['total_gt_meta'] - res['total_gt_ENE'])
res = calc_perf(df_rd5_tcga, df_tcga, type='tcga')
print("TCGA meta sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['meta_sens'], res['meta_spec'], res['meta_f1'], res['meta_acc']))
calc_physician_auc(res['meta_sens'], res['meta_spec'], res['total_gt_meta'], res['total_neg_meta'])
print("TCGA ENE sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['ene_sens'], res['ene_spec'], res['ene_f1'], res['ene_acc']))
calc_physician_auc(res['ene_sens'], res['ene_spec'], res['total_gt_ENE'], res['total_neg_ENE'])

print("Reader6-Albert:")
res = calc_perf(df_rd6_renji, df_renji, type='renji')
print("Renji meta sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['meta_sens'], res['meta_spec'], res['meta_f1'], res['meta_acc']))
calc_physician_auc(res['meta_sens'], res['meta_spec'], res['total_gt_meta'], res['total_neg_meta'])
print("Renji ENE sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['ene_sens'], res['ene_spec'], res['ene_f1'], res['ene_acc']))
print("ENE+ Vs. ENE- sensitivity {:.4f}, specificity {:.4f}, acc {:.4f}".format(res['enevspos_sens'], res['enevspos_spec'], res['enevspos_acc']))
calc_physician_auc(res['ene_sens'], res['ene_spec'], res['total_gt_ENE'], res['total_neg_ENE'])
calc_physician_auc(res['enevspos_sens'], res['enevspos_spec'], res['total_gt_ENE'], res['total_gt_meta'] - res['total_gt_ENE'])
res = calc_perf(df_rd6_cgmh, df_cgmh, type='cgmh')
print("CGMH meta sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['meta_sens'], res['meta_spec'], res['meta_f1'], res['meta_acc']))
calc_physician_auc(res['meta_sens'], res['meta_spec'], res['total_gt_meta'], res['total_neg_meta'])
print("CGMH ENE sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['ene_sens'], res['ene_spec'], res['ene_f1'], res['ene_acc']))
print("ENE+ Vs. ENE- sensitivity {:.4f}, specificity {:.4f}, acc {:.4f}".format(res['enevspos_sens'], res['enevspos_spec'], res['enevspos_acc']))
calc_physician_auc(res['ene_sens'], res['ene_spec'], res['total_gt_ENE'], res['total_neg_ENE'])
calc_physician_auc(res['enevspos_sens'], res['enevspos_spec'], res['total_gt_ENE'], res['total_gt_meta'] - res['total_gt_ENE'])
res = calc_perf(df_rd6_tcga, df_tcga, type='tcga')
print("TCGA meta sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['meta_sens'], res['meta_spec'], res['meta_f1'], res['meta_acc']))
calc_physician_auc(res['meta_sens'], res['meta_spec'], res['total_gt_meta'], res['total_neg_meta'])
print("TCGA ENE sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['ene_sens'], res['ene_spec'], res['ene_f1'], res['ene_acc']))
calc_physician_auc(res['ene_sens'], res['ene_spec'], res['total_gt_ENE'], res['total_neg_ENE'])

Reader1-WJ:
Renji meta sensitivity 0.8293, specificity 0.7672, f1 0.6667, acc 0.7834
AUC: 0.7982478042281782 (0.7478812538466751-0.8486143546096813)
Renji ENE sensitivity 0.5484, specificity 0.9659, f1 0.5397, acc 0.9384
ENE+ Vs. ENE- sensitivity 0.5484, specificity 0.8370, acc 0.7642
AUC: 0.7569613387084073 (0.6561878917174925-0.8577347856993222)
AUC: 0.6926543248246971 (0.5786946409419816-0.8066140087074126)
CGMH meta sensitivity 0.6667, specificity 0.8571, f1 0.5641, acc 0.8247
AUC: 0.7618780295558383 (0.661808231898886-0.8619478272127905)
CGMH ENE sensitivity 0.1739, specificity 1.0000, f1 0.2963, acc 0.9021
ENE+ Vs. ENE- sensitivity 0.1739, specificity 1.0000, acc 0.4242
AUC: 0.5860781669011054 (0.4570541907693952-0.7151021430328155)
AUC: 0.5860781321475627 (0.3772208255026287-0.7949354387924966)
TCGA meta sensitivity 0.5962, specificity 0.9027, f1 0.5905, acc 0.8453
AUC: 0.7493373999630402 (0.6680571907559396-0.8306176091701407)
TCGA ENE sensitivity 0.1765, specificity 0.9885, f1

In [29]:
print("Reader1-WJ:")
res = calc_patient_level_perf(df_rd1_renji, df_renji, type='renji')
r1_renji_ene_pred = res['patient_ene_preds']
r1_renji_ene_label = res['anno_ene']
print("Renji meta sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['meta_sens'], res['meta_spec'], res['meta_f1'], res['meta_acc']))
calc_physician_auc(res['meta_sens'], res['meta_spec'], res['total_gt_meta'], res['total_neg_meta'])
print("Renji ENE sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['ene_sens'], res['ene_spec'], res['ene_f1'], res['ene_acc']))
calc_physician_auc(res['ene_sens'], res['ene_spec'], res['total_gt_ENE'], res['total_neg_ENE'])
res = calc_patient_level_perf(df_rd1_cgmh, df_cgmh, type='cgmh')
r1_cgmh_ene_pred = res['patient_ene_preds']
r1_cgmh_ene_label = res['anno_ene']
print("CGMH meta sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['meta_sens'], res['meta_spec'], res['meta_f1'], res['meta_acc']))
calc_physician_auc(res['meta_sens'], res['meta_spec'], res['total_gt_meta'], res['total_neg_meta'])
print("CGMH ENE sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['ene_sens'], res['ene_spec'], res['ene_f1'], res['ene_acc']))
calc_physician_auc(res['ene_sens'], res['ene_spec'], res['total_gt_ENE'], res['total_neg_ENE'])
res = calc_patient_level_perf(df_rd1_tcga, df_tcga, type='tcga')
print("TCGA meta sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['meta_sens'], res['meta_spec'], res['meta_f1'], res['meta_acc']))
calc_physician_auc(res['meta_sens'], res['meta_spec'], res['total_gt_meta'], res['total_neg_meta'])
# print("TCGA ENE sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['ene_sens'], res['ene_spec'], res['ene_f1'], res['ene_acc']))
# calc_physician_auc(res['ene_sens'], res['ene_spec'], res['total_gt_ENE'], res['total_neg_ENE'])

print("Reader2-Zhang:")
res = calc_patient_level_perf(df_rd2_renji, df_renji, type='renji')
r2_renji_ene_pred = res['patient_ene_preds']
r2_renji_ene_label = res['anno_ene']
print("Renji meta sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['meta_sens'], res['meta_spec'], res['meta_f1'], res['meta_acc']))
calc_physician_auc(res['meta_sens'], res['meta_spec'], res['total_gt_meta'], res['total_neg_meta'])
print("Renji ENE sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['ene_sens'], res['ene_spec'], res['ene_f1'], res['ene_acc']))
calc_physician_auc(res['ene_sens'], res['ene_spec'], res['total_gt_ENE'], res['total_neg_ENE'])
res = calc_patient_level_perf(df_rd2_cgmh, df_cgmh, type='cgmh')
r2_cgmh_ene_pred = res['patient_ene_preds']
r2_cgmh_ene_label = res['anno_ene']
print("CGMH meta sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['meta_sens'], res['meta_spec'], res['meta_f1'], res['meta_acc']))
calc_physician_auc(res['meta_sens'], res['meta_spec'], res['total_gt_meta'], res['total_neg_meta'])
print("CGMH ENE sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['ene_sens'], res['ene_spec'], res['ene_f1'], res['ene_acc']))
calc_physician_auc(res['ene_sens'], res['ene_spec'], res['total_gt_ENE'], res['total_neg_ENE'])
res = calc_patient_level_perf(df_rd2_tcga, df_tcga, type='tcga')
print("TCGA meta sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['meta_sens'], res['meta_spec'], res['meta_f1'], res['meta_acc']))
calc_physician_auc(res['meta_sens'], res['meta_spec'], res['total_gt_meta'], res['total_neg_meta'])
# print("TCGA ENE sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['ene_sens'], res['ene_spec'], res['ene_f1'], res['ene_acc']))
# calc_physician_auc(res['ene_sens'], res['ene_spec'], res['total_gt_ENE'], res['total_neg_ENE'])

print("Reader3-DD:")
res = calc_patient_level_perf(df_rd4_renji, df_renji, type='renji')
r3_renji_ene_pred = res['patient_ene_preds']
r3_renji_ene_label = res['anno_ene']
print("Renji meta sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['meta_sens'], res['meta_spec'], res['meta_f1'], res['meta_acc']))
calc_physician_auc(res['meta_sens'], res['meta_spec'], res['total_gt_meta'], res['total_neg_meta'])
print("Renji ENE sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['ene_sens'], res['ene_spec'], res['ene_f1'], res['ene_acc']))
calc_physician_auc(res['ene_sens'], res['ene_spec'], res['total_gt_ENE'], res['total_neg_ENE'])
res = calc_patient_level_perf(df_rd4_cgmh, df_cgmh, type='cgmh')
r3_cgmh_ene_pred = res['patient_ene_preds']
r3_cgmh_ene_label = res['anno_ene']
print("CGMH meta sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['meta_sens'], res['meta_spec'], res['meta_f1'], res['meta_acc']))
calc_physician_auc(res['meta_sens'], res['meta_spec'], res['total_gt_meta'], res['total_neg_meta'])
print("CGMH ENE sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['ene_sens'], res['ene_spec'], res['ene_f1'], res['ene_acc']))
calc_physician_auc(res['ene_sens'], res['ene_spec'], res['total_gt_ENE'], res['total_neg_ENE'])
res = calc_patient_level_perf(df_rd4_tcga, df_tcga, type='tcga')
print("TCGA meta sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['meta_sens'], res['meta_spec'], res['meta_f1'], res['meta_acc']))
calc_physician_auc(res['meta_sens'], res['meta_spec'], res['total_gt_meta'], res['total_neg_meta'])
# print("TCGA ENE sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['ene_sens'], res['ene_spec'], res['ene_f1'], res['ene_acc']))
# calc_physician_auc(res['ene_sens'], res['ene_spec'], res['total_gt_ENE'], res['total_neg_ENE'])

print("Reader4-XBB:")
res = calc_patient_level_perf(df_rd5_renji, df_renji, type='renji')
r4_renji_ene_pred = res['patient_ene_preds']
r4_renji_ene_label = res['anno_ene']
print("Renji meta sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['meta_sens'], res['meta_spec'], res['meta_f1'], res['meta_acc']))
calc_physician_auc(res['meta_sens'], res['meta_spec'], res['total_gt_meta'], res['total_neg_meta'])
print("Renji ENE sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['ene_sens'], res['ene_spec'], res['ene_f1'], res['ene_acc']))
calc_physician_auc(res['ene_sens'], res['ene_spec'], res['total_gt_ENE'], res['total_neg_ENE'])
res = calc_patient_level_perf(df_rd5_cgmh, df_cgmh, type='cgmh')
r4_cgmh_ene_pred = res['patient_ene_preds']
r4_cgmh_ene_label = res['anno_ene']
print("CGMH meta sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['meta_sens'], res['meta_spec'], res['meta_f1'], res['meta_acc']))
calc_physician_auc(res['meta_sens'], res['meta_spec'], res['total_gt_meta'], res['total_neg_meta'])
print("CGMH ENE sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['ene_sens'], res['ene_spec'], res['ene_f1'], res['ene_acc']))
calc_physician_auc(res['ene_sens'], res['ene_spec'], res['total_gt_ENE'], res['total_neg_ENE'])
res = calc_patient_level_perf(df_rd5_tcga, df_tcga, type='tcga')
print("TCGA meta sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['meta_sens'], res['meta_spec'], res['meta_f1'], res['meta_acc']))
calc_physician_auc(res['meta_sens'], res['meta_spec'], res['total_gt_meta'], res['total_neg_meta'])
# print("TCGA ENE sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['ene_sens'], res['ene_spec'], res['ene_f1'], res['ene_acc']))
# calc_physician_auc(res['ene_sens'], res['ene_spec'], res['total_gt_ENE'], res['total_neg_ENE'])

print("Reader6-Albert:")
res = calc_patient_level_perf(df_rd6_renji, df_renji, type='renji')
r5_renji_ene_pred = res['patient_ene_preds']
r5_renji_ene_label = res['anno_ene']
print("Renji meta sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['meta_sens'], res['meta_spec'], res['meta_f1'], res['meta_acc']))
calc_physician_auc(res['meta_sens'], res['meta_spec'], res['total_gt_meta'], res['total_neg_meta'])
print("Renji ENE sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['ene_sens'], res['ene_spec'], res['ene_f1'], res['ene_acc']))
calc_physician_auc(res['ene_sens'], res['ene_spec'], res['total_gt_ENE'], res['total_neg_ENE'])
res = calc_patient_level_perf(df_rd6_cgmh, df_cgmh, type='cgmh')
r5_cgmh_ene_pred = res['patient_ene_preds']
r5_cgmh_ene_label = res['anno_ene']
print("CGMH meta sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['meta_sens'], res['meta_spec'], res['meta_f1'], res['meta_acc']))
calc_physician_auc(res['meta_sens'], res['meta_spec'], res['total_gt_meta'], res['total_neg_meta'])
print("CGMH ENE sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['ene_sens'], res['ene_spec'], res['ene_f1'], res['ene_acc']))
calc_physician_auc(res['ene_sens'], res['ene_spec'], res['total_gt_ENE'], res['total_neg_ENE'])
res = calc_patient_level_perf(df_rd6_tcga, df_tcga, type='tcga')
print("TCGA meta sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['meta_sens'], res['meta_spec'], res['meta_f1'], res['meta_acc']))
calc_physician_auc(res['meta_sens'], res['meta_spec'], res['total_gt_meta'], res['total_neg_meta'])
# print("TCGA ENE sensitivity {:.4f}, specificity {:.4f}, f1 {:.4f}, acc {:.4f}".format(res['ene_sens'], res['ene_spec'], res['ene_f1'], res['ene_acc']))
# calc_physician_auc(res['ene_sens'], res['ene_spec'], res['total_gt_ENE'], res['total_neg_ENE'])

Reader1-WJ:
Renji meta sensitivity 0.9796, specificity 1.0000, f1 0.9897, acc 0.9800
AUC: 0.9897959083715121 (0.9555071947964493-1.024084621946575)
Renji ENE sensitivity 0.8095, specificity 0.7241, f1 0.7391, acc 0.7600
AUC: 0.7668016689021875 (0.628596095874074-0.905007241930301)
CGMH meta sensitivity 1.0000, specificity 0.4167, f1 0.7879, acc 0.7200
AUC: 0.7083168971311545 (0.5031881126247623-0.9134456816375468)
CGMH ENE sensitivity 0.3636, specificity 1.0000, f1 0.5333, acc 0.7200
AUC: 0.6799815956674184 (0.46284256194980705-0.8971206293850298)
TCGA meta sensitivity 0.8571, specificity 0.3333, f1 0.5714, acc 0.5263
AUC: 0.5952380340136141 (0.32084783543845746-0.8696282325887708)
Reader2-Zhang:
Renji meta sensitivity 0.9796, specificity 1.0000, f1 0.9897, acc 0.9800
AUC: 0.9897959083715121 (0.9555071947964493-1.024084621946575)
Renji ENE sensitivity 0.7143, specificity 0.6552, f1 0.6522, acc 0.6800
AUC: 0.6847191156131176 (0.5317928768636414-0.8376453543625938)
CGMH meta sensitivity 

TCGA meta sensitivity 0.8571, specificity 0.4167, f1 0.6000, acc 0.5789
AUC: 0.6368939250249734 (0.3671836196488893-0.9066042304010575)
Reader3-DD:
Renji meta sensitivity 1.0000, specificity 1.0000, f1 1.0000, acc 1.0000
AUC: 0.9999999897959185 (0.9999717060447727-1.0000282735470643)
Renji ENE sensitivity 1.0000, specificity 0.4483, f1 0.7241, acc 0.6800
AUC: 0.7241161301148231 (0.577390356405233-0.8708419038244133)
CGMH meta sensitivity 1.0000, specificity 0.0833, f1 0.7027, acc 0.5600
AUC: 0.5416561932869731 (0.31187880214232655-0.7714335844316197)
CGMH ENE sensitivity 0.6364, specificity 0.9286, f1 0.7368, acc 0.8000
AUC: 0.7824386476873515 (0.5925291611807343-0.9723481341939687)
TCGA meta sensitivity 1.0000, specificity 0.0000, f1 0.5385, acc 0.3684
AUC: 0.4999999292929393 (0.2239212824382804-0.7760785761475981)
Reader4-XBB:
Renji meta sensitivity 1.0000, specificity 1.0000, f1 1.0000, acc 1.0000
AUC: 0.9999999897959185 (0.9999717060447727-1.0000282735470643)
Renji ENE sensitivity 

In [30]:
import statsmodels.stats.inter_rater as irr
from statsmodels.stats.inter_rater import aggregate_raters

renji_ene_rater_all = np.stack([r1_renji_ene_pred, r2_renji_ene_pred, r3_renji_ene_pred,r4_renji_ene_pred,r5_renji_ene_pred], axis=1)
print(renji_ene_rater_all.shape)
intermediate = aggregate_raters(renji_ene_rater_all)
renji_ene_fleiss_kappa = irr.fleiss_kappa(intermediate[0], "fleiss")
print("EENT ENE Fleiss Kappa: ", renji_ene_fleiss_kappa)


(50, 5)
EENT ENE Fleiss Kappa:  0.19643509387099134


In [31]:
import statsmodels.stats.inter_rater as irr
from statsmodels.stats.inter_rater import aggregate_raters

cgmh_ene_rater_all = np.stack([r1_cgmh_ene_pred, r2_cgmh_ene_pred, r3_cgmh_ene_pred,r4_cgmh_ene_pred,r5_cgmh_ene_pred], axis=1)
print(cgmh_ene_rater_all.shape)
intermediate = aggregate_raters(cgmh_ene_rater_all)
cgmh_ene_fleiss_kappa = irr.fleiss_kappa(intermediate[0], "fleiss")
print("CGMH ENE Fleiss Kappa: ", cgmh_ene_fleiss_kappa)


(25, 5)
CGMH ENE Fleiss Kappa:  0.4150595575723198
